# Usage

In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import scipy
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import skimage
import skimage.io
import seaborn as sb
import tifffile

from nima import nima

cm = plt.cm.inferno_r

fp = "../../tests/data/1b_c16_15.tif"

channels = ["G", "R", "C"]
# dark = io.imread('/home/dati/GBM_persson/analyses/15.02.05_cal-GBM5-pBJclop/dark/dark-25_500.tif')
# flat = io.imread('/home/dati/GBM_persson/analyses/15.02.05_cal-GBM5-pBJclop/flat/flat-C-dark-37bis_500.tif')

In [ ]:
d_im, n_ch, times = nima.read_tiff(fp, channels)

In [ ]:
d_im_bg, bgs, ff, bgv = nima.d_bg(d_im, downscale=(2, 2), kind="li_adaptive")
bgs

In [ ]:
d_im_bg["G"].__sizeof__()

In [ ]:
scipy.stats.distributions.norm.fit(bgv["C"][1])

In [ ]:
sb.histplot(bgv["C"][1], kde=True, stat="density")
plt.yscale("log")

In [ ]:
ff["C"][1][0]

In [ ]:
for i in range(4):
    plt.hist(bgv["C"][i], histtype="step", bins=32, log=1)

In [ ]:
nima.d_mask_label(
    d_im_bg, threshold_method="yen", min_size=2000, channels=channels, watershed=0
)
d_im_bg.keys()

In [ ]:
f = nima.d_show(d_im_bg)

In [ ]:
meas, pr = nima.d_meas_props(d_im_bg)
f = nima.d_plot_meas(bgs, meas, channels=channels)

In [ ]:
plt.subplot(1, 2, 1)
skimage.io.imshow(d_im_bg["r_cl"][2], vmin=0.0, vmax=1.1)
plt.subplot(1, 2, 2)
skimage.io.imshow(d_im_bg["r_pH"][2], vmin=7.3, vmax=10.7)

In [ ]:
c0 = d_im["C"][0]
g0 = d_im["G"][0]
r0 = d_im["R"][0]
tifffile.imshow(1 / c0 / c0)

In [ ]:
background_channel1 = skimage.restoration.rolling_ball(c0, radius=160)
tifffile.imshow(background_channel1)

In [ ]:
plt.plot(background_channel1[200])

## background, AF, target cells

In [ ]:
import aicsimageio

In [ ]:
aicsim = aicsimageio.AICSImage(
    "/home/dati/GBM_persson/data/15.02.05_cal-GBM5-pBJclop/ph633/1_20_40.tif"
)

In [ ]:
aicsim.ome_metadata

In [ ]:
import nima_io.read as ir

In [ ]:
md, wr = ir.read(
    "/home/dati/GBM_persson/data/15.02.05_cal-GBM5-pBJclop/ph633/1_20_40.tif"
)

In [ ]:
md.full

In [ ]:
channels3 = ["G", "R", "C", "G2"]
dim3, _, _ = nima.read_tiff(
    "/home/dati/GBM_persson/data/15.02.05_cal-GBM5-pBJclop/ph633/1_20_40.tif", channels3
)
dim3, bgm3, _, bgv3 = nima.d_bg(dim3, downscale=(2, 2), kind="li_adaptive")
c = dim3["C"]
g = dim3["G"]
r = dim3["R"]

bc = bgv3["C"][3]
bg = bgv3["G"][3]
br = bgv3["R"][3]

c = d_im_bg['C']
g = d_im_bg['G']
r = d_im_bg['R']

bc = bgv['C'][2]
bg = bgv['G'][3]
br = bgv['R'][3]

In [ ]:
skimage.io.imshow(dim3["R"][1])

In [ ]:
plt.plot(dim3["R"][1][80, 30:])

In [ ]:
mmin = min(len(bc), len(bg), len(br))

print("c =", len(bc))
print("g =", len(bg))
print("r =", len(br))

plt.figure(figsize=(7.5, 6))

plt.subplot(2, 2, 1)
mmin = min(len(bc), len(bg))
plt.hexbin(bg[:mmin], bc[:mmin], bins="log", cmap=plt.cm.viridis_r)
cb = plt.colorbar()
plt.xlabel("green")
plt.ylabel("cyan")

plt.subplot(2, 2, 2)
mmin = min(len(bc), len(br))
plt.hexbin(br[:mmin], bc[:mmin], bins="log", cmap=plt.cm.viridis_r)
cb = plt.colorbar()
plt.xlabel("red")
plt.ylabel("cyan")

ax = plt.subplot(2, 2, 4)
mmin = min(len(br), len(bg))
plt.hexbin(br[:mmin], bg[:mmin], bins="log", cmap=plt.cm.viridis_r)
cb = plt.colorbar()
plt.xlabel("red")
plt.ylabel("green")

In [ ]:
from mpl_toolkits.axes_grid1.inset_locator import mark_inset

plt.figure(figsize=(10, 8))
plt.subplot(2, 2, 1)
plt.hexbin(g.ravel(), c.ravel(), bins="log", cmap=plt.cm.viridis_r)
cb = plt.colorbar()
plt.xlabel("green")
plt.ylabel("cyan")

plt.subplot(2, 2, 2)
plt.hexbin(r.ravel(), c.ravel(), bins="log", cmap=plt.cm.viridis_r)
cb = plt.colorbar()
plt.xlabel("red")
plt.ylabel("cyan")

ax = plt.subplot(2, 2, 4)
plt.hexbin(r.ravel(), g.ravel(), bins="log", cmap=plt.cm.viridis_r)
cb = plt.colorbar()
plt.xlabel("red")
plt.ylabel("green")

axins = plt.axes([0.2, 0.12, 0.28, 0.28])
axins.hexbin(
    r.ravel(), g.ravel(), extent=(0, 80, 0, 150), bins="log", cmap=plt.cm.viridis_r
)

mark_inset(ax, axins, loc1=1, loc2=4, fc="none", ec="0.5")

In [ ]:
import statsmodels.api as sm


# import pylab
def qqplot(v1, v2):
    ml = min(len(v1), len(v2))
    sm.qqplot_2samples(v1[:ml], v2[:ml])
    # pylab.show()


qqplot(bgv3["R"][2], bgv3["G"][2])
qqplot(bgv3["C"][3], bgv3["R"][1])

## flat image correction

In [ ]:
g1a = nima.read_tiff("/home/dati/dt-clop3/data/210917/3alpha/c1_2.tf8", channels)[0][
    "G"
][2]
g2a = nima.read_tiff("/home/dati/dt-clop3/data/210917/3alpha/c2_2.tf8", channels)[0][
    "G"
][2]
g3a = nima.read_tiff("/home/dati/dt-clop3/data/210917/3alpha/c3_2.tf8", channels)[0][
    "G"
][2]
g1 = nima.read_tiff("/home/dati/dt-clop3/data/210917/3/c1_2.tf8", channels)[0]["G"][2]
g2 = nima.read_tiff("/home/dati/dt-clop3/data/210917/3/c2_2.tf8", channels)[0]["G"][2]
g3 = nima.read_tiff("/home/dati/dt-clop3/data/210917/3/c3_2.tf8", channels)[0]["G"][2]

In [ ]:
plt.subplot(1, 2, 1)
skimage.io.imshow(
    skimage.filters.gaussian(np.max([g1, g2, g3, g1a, g2a, g3a], axis=0), sigma=500)
)
plt.subplot(1, 2, 2)
skimage.io.imshow(skimage.filters.gaussian(np.max([g1, g2, g3], axis=0), sigma=500))

Run simulazion to identify best algorithm for FLAT calculation.

(Do we need to simulate a fluorophore solution sample?)

Compare to external image simulator.

In [ ]:
phantom = skimage.io.imread(
    "/home/dan/2024-02-05_15h-38m-58s_image_all_tif/final/final.tif"
)

In [ ]:
skimage.io.imshow(phantom[14])

In [ ]:
sb.histplot(phantom[1].flatten(), kde=True)
plt.yscale("log")

## utils.bg

In [ ]:
mmap24 = tifffile.memmap(
    "/home/dati/dt-clop3/data/210917/3/c1_2.tf8", mode="r", page=24
)
mmap25 = tifffile.memmap(
    "/home/dati/dt-clop3/data/210917/3/c1_2.tf8", mode="r", page=25
)
mmap26 = tifffile.memmap(
    "/home/dati/dt-clop3/data/210917/3/c1_2.tf8", mode="r", page=26
)
skimage.io.imshow(mmap25)

In [ ]:
sb.histplot(mmap24.ravel())
plt.yscale("log")

In [ ]:
from nima import utils

In [ ]:
bg24, sd24 = utils.bg(mmap24, bgmax=114950000000000000000000000000)
bg25, sd25 = utils.bg(mmap25)
bg26, sd26 = utils.bg(mmap26)
bg24, sd24, bg25, sd25, bg26, sd26

In [ ]:
sb.histplot(mmap25[(utils.prob(mmap25, bg25, sd25) > 0.00033)])

In [ ]:
skimage.io.imshow(utils.prob(mmap25, bg25, sd25) > 0.033)

In [ ]:
bg = mmap24[(utils.prob(mmap24, bg24, sd24) > 0.00033)]
br = mmap25[(utils.prob(mmap25, bg25, sd25) > 0.00000033)]
bc = mmap26[(utils.prob(mmap26, bg26, sd26) > 0.00000033)]

mmin = min(len(bc), len(bg), len(br))

print("c =", len(bc))
print("g =", len(bg))
print("r =", len(br))

plt.figure(figsize=(7.5, 6))

plt.subplot(2, 2, 1)
mmin = min(len(bc), len(bg))
plt.hexbin(bg[:mmin], bc[:mmin], bins="log", cmap=plt.cm.viridis_r)
cb = plt.colorbar()
plt.xlabel("green")
plt.ylabel("cyan")

plt.subplot(2, 2, 2)
mmin = min(len(bc), len(br))
plt.hexbin(br[:mmin], bc[:mmin], bins="log", cmap=plt.cm.viridis_r)
cb = plt.colorbar()
plt.xlabel("red")
plt.ylabel("cyan")

ax = plt.subplot(2, 2, 4)
mmin = min(len(br), len(bg))
plt.hexbin(br[:mmin], bg[:mmin], bins="log", cmap=plt.cm.viridis_r)
cb = plt.colorbar()
plt.xlabel("red")
plt.ylabel("green")

In [ ]:
with tifffile.TiffFile("/home/dati/dt-clop3/data/210917/3alpha/c1_2.tf8") as tif:
    im = tif.asarray()
    axes = tif.series[0].axes
    print(axes)

## BIAS and FLAT

TODO:
- pytest
- build a function callable from both the library and the CLI